This notebook is for running NAQS on molecules and reproduction of experimental results/baslines.  It is split into two sections, ***which will set up different environments and so will require the runtime resetting if you want to switch***.

First, however, we'll just pull down and step into the repo itself.

In [1]:
!git clone https://github.com/maxleitner/naqs-for-quantum-chemistry.git

import os
os.chdir("naqs-for-quantum-chemistry")

Cloning into 'naqs-for-quantum-chemistry'...
remote: Enumerating objects: 248, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 248 (delta 28), reused 44 (delta 21), pack-reused 190 (from 1)
Receiving objects: 100% (248/248), 26.87 MiB | 18.60 MiB/s, done.
Resolving deltas: 100% (62/62), done.
Updating files: 100% (139/139), done.


# 1. Solve molecules using NAQS

To run this section, we first install OpenFermion and build the necessary cython scripts.

In [2]:
!chmod -R u+x experiments/bash/naqs

try:
    import openfermion as of
except:
    !python -m pip install openfermion==0.11.0
    !python -m pip install openfermionpsi4

!python src_cpp/setup.py build_ext --inplace --force

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.4/631.4 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for openfermion: filename=openfermion-0.11.0-py3-none-any.whl size=741932 sha256=0a4654a6930fb0a3d2592a62a4e9ee4826d4079ec6bcb8fe4c228b26ea8eeafa
  Stored in directory: /root/.cache/pip/wheels/da/8a/ed/eab45c8ff1021cde17066d8315e49d9341c8639a88cc8e2b25
  Created wheel for pubchempy: filename=PubChemPy-1.0.4-py3-none-any.whl size=13818 sha256=d5c1740cb816e23a54ce55c17a9e0170f6d5bd5ea60849d14ced38f5108645b7
  Stored in directory: /root/.cache/pip/wheels/8b/e3/6c/3385b2db08b0985a87f5b117f98d0cb61a3ae3ca3bcbbd8307
Successfully built openfermion pubchempy
Compiling src_cpp/sparse_math.pyx because it changed.
Compiling src_cpp/hilbert_math.pyx because it changed.
Compiling src_cpp/hamiltonian_math.pyx because it changed.
[1/3] Cythonizing src_cpp/hamiltonian_math.pyx
/usr/local/lib/python3.11/dist-packages/Cython/Co

Behind the scence, the script to run specific experiments is ``experiments.run``, however there are many ways to configure this, some of which are not discussed in the paper.  For completeness, the optional arguments can be viewed by running with a ``-h`` or ``--help`` flag.

In [3]:
!python -u -m experiments.run -h

usage: run.py [-h] [-m [MOLECULE]] [-hf [HAMILTONIAN_FNAME]] [-o [OUT]]
              [-n [NUMBER]] [-qo [QUBIT_ORDERING]] [-l [LOAD]] [-c] [-r]
              [-n_samps [N_SAMPS]] [-n_samps_max [N_SAMPS_MAX]]
              [-n_unq_samps_max [N_UNQ_SAMPS_MAX]]
              [-n_unq_samps_min [N_UNQ_SAMPS_MIN]] [-weight_by_psi]
              [-no_mask_psi] [-full_mask_psi] [-lr [LR]] [-lr_lut [LR_LUT]]
              [-n_train [N_TRAIN]] [-n_pretrain [N_PRETRAIN]] [-n_lut [N_LUT]]
              [-n_hid [N_HID]] [-n_layer [N_LAYER]]
              [-n_hid_phase [N_HID_PHASE]] [-n_layer_phase [N_LAYER_PHASE]]
              [-output_freq [OUTPUT_FREQ]] [-save_freq [SAVE_FREQ]] [-loadH]
              [-overwriteH] [-presolveH]
              [-n_excitations_max [N_EXCITATIONS_MAX]] [-comb_amp_phase]
              [-no_amp_sym] [-phase_sym] [-single_phase] [-no_restrictedH]
              [-v] [-s [SEED]]

Run experimental script.

options:
  -h, --help            show this help message and exit


An example of a configured script, would be this.

In [14]:
!python -u -m experiments.run -o "data/LiH" -m "molecules/LiH" -single_phase -n1 -n_layer 1 -n_hid 16 -n_layer_phase 1 -n_hid_phase 32 -lr 0.001 -s 1 -n_train 3000 -output_freq 25 -save_freq -1

Running experimental script: /content/naqs-for-quantum-chemistry/experiments/_base.py
Results will be saved to: data/LiH/

script options:
	molecule_fname : molecules/LiH
	hamiltonian_fname : None
	exp_name : data/LiH
	num_experiments : 1
	load (pre_trained model) : None
	continue_experiment : False
	qubit_ordering : -1
	lr : 0.001
	lr_lut : 0.01
	n_samps : 10000000.0
	n_samps_max : 1000000000000
	n_unq_samps_max : 100000
	n_unq_samps_min : 10000
	weight_by_psi : False
	no_mask_psi : False
	full_mask_psi : False
	n_train : 3000
	n_pretrain : 0
	n_lut : 0
	n_hid : 16
	n_layer : 1
	n_hid_phase : 32
	n_layer_phase : 1
	output_freq : 25
	save_freq : -1
	comb_amp_phase : False
	no_amp_sym : False
	phase_sym : False
	single_phase : True
	loadH : False
	overwriteH : False
	presolveH : False
	n_excitations_max : -1
	no_restrictedH : False
	reset_optimizer : False
	verbose : False
	seed : 1


------------------------------------------
	Setting global seed using 1.
------------------------------

To ***reproduce the results reported in the paper***, batch training scripts which will perform 5 optimisations with different seeds using all the correct hyperparemeters are provided.  Each script takes the GPU number to use and the molecule name, corresponding to the a sub-folder in ``molecules``.

In [15]:
!experiments/bash/naqs/batch_train.sh 0 LiH
# !experiments/bash/naqs/batch_train_no_amp_sym.sh 0 LiH
# !experiments/bash/naqs/batch_train_no_mask.sh 0 LiH
# !experiments/bash/naqs/batch_train_full_mask.sh 0 LiH

GPU_NUM  = 0
MOLECULE NAME  = LiH
running exp 1/5...output can be viewed at LiH_s111.out
nohup: ignoring input and redirecting stderr to stdout
running exp 2/5...output can be viewed at LiH_s222.out
nohup: ignoring input and redirecting stderr to stdout
running exp 3/5...output can be viewed at LiH_s333.out
nohup: ignoring input and redirecting stderr to stdout
running exp 4/5...output can be viewed at LiH_s444.out
nohup: ignoring input and redirecting stderr to stdout
running exp 5/5...output can be viewed at LiH_s555.out
nohup: ignoring input and redirecting stderr to stdout


# 2. Solve molecules using Psi4 (baseline calculations)

First, install Psi4 into a miniconda environment.

In [16]:
import sys

# Conda env creation and Psi 4 installation.
!wget https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!bash Miniconda3-4.5.4-Linux-x86_64.sh -bfp /usr/local
sys.path.append('/usr/local/lib/python3.6/site-packages')
!conda install -c psi4 psi4 -y

--2025-06-27 16:24:42--  https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.176.84, 104.18.177.84, 2606:4700::6812:b154, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.176.84|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh [following]
--2025-06-27 16:24:42--  https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.191.158, 104.16.32.241, 2606:4700::6810:20f1, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.191.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58468498 (56M) [application/x-sh]
Saving to: ‘Miniconda3-4.5.4-Linux-x86_64.sh’

Miniconda3-4.5.4-Li 100%[===================>]  55.76M   176MB/s    in 0.3s    

2025-06-27 16:24:42 (176 MB/s) - ‘Miniconda3-4.5.4-Linux

Then run pre-prepared scripts.

In [17]:
!psi4 experiments/bash/psi4/LiH.dat


Results for LiH.dat

Geometry : 
	Li 3 0 0
	H 2 0 0

HF :  -7.767362
CCSD :  -7.784455
CCSD(T) :  -7.784460
FCI :  -7.784460
